In [ ]:
import pandas as pd
import glob

import csv
import matplotlib.pyplot as plt


In [ ]:
import os
current_directory = os.getcwd()
print("Current directory : ", current_directory)

aymeric =  "/home/aymeric/python-scripts/anses_medialab/datas/" #aymeric
jp = '~/Dropbox/Mac/Desktop/CRD Anses/all3/' # Jean Philippe
jp_index = '~/Dropbox/Mac/Desktop/CRD Anses/code/indexation_results/' # Jean Philippe index

if 'aymeric' in current_directory:
    path_base = aymeric

elif 'Mac' in current_directory:
    path_base = jp
elif 'd:/Projects' in current_directory:
    path_base = "d:/Projects/Medialab/"

print("Path base : ", path_base)

In [ ]:
import glob
tweet_path = f"{path_base}tweets_pesticides/tweets_pesticides and cie/"
paths=glob.glob(tweet_path)
paths
i=-1
dic_id={}
for x in [x for x in pd.read_csv(glob.glob(f'{tweet_path}*.csv')[0]).columns if 'id' in x]:
    dic_id[x]=str
dic_id
for name in glob.glob(f'{tweet_path}*.csv'):
    print (name)
    i+=1
    df0=pd.read_csv(name, dtype=dic_id, usecols=['id','user_id', 'text', 'user_screen_name', 'to_userid',
       'to_tweetid','mentioned_names', 'mentioned_ids','local_time'])
    
    if i>0:
        df=pd.concat([df,df0])
    else:
        df=df0


In [ ]:

tweets= df.drop_duplicates()
tweets['local_time'] = pd.to_datetime(tweets['local_time'])
#tweets


print (len(tweets))

In [ ]:
tweets.loc[tweets.user_id=="1125791906548592642"]

In [ ]:
# Using '*' pattern 
#print('\nNamed with wildcard *:')
i=-1
dic_id={}
for x in [x for x in pd.read_csv(f"{path_base}data_for_industry/20250902_pesticides_industries.csv").columns if 'id' in x]:
    dic_id[x]=str
dic_id
users0 = pd.read_csv(f"{path_base}data_for_industry/20250902_pesticides_industries.csv", sep =",", dtype=dic_id)

dict_col = {}
for x in ['field_work_x', 'metier_x', 'Ingenieur_x', 'Agronome_x',
       'industry_verif_x', 'field_work_y', 'metier_y', 'Ingenieur_y',
       'Agronome_y', 'industry_verif_y']:
    dict_col[x] = x.replace("_x","").replace('_y',"")
dict_col
users0 = users0[['user_id', 'user_screen_name', 'user_description',
       'account_publication', 'industry', 'Type_entite', 'company', 'UIPP',
       'field_work_x', 'metier_x', 'Ingenieur_x', 'Agronome_x',
       'industry_verif_x']].rename(columns=dict_col)
users0

In [ ]:
# Using '*' pattern 
#print('\nNamed with wildcard *:')
i=-1
dic_id={}
for x in [x for x in pd.read_csv(f'{path_base}the_true_all_annotated_account.csv').columns if 'id' in x]:
    dic_id[x]=str
dic_id
users1 = pd.read_csv(f'{path_base}the_true_all_annotated_account.csv', sep =",", dtype=dic_id)

users1["user_description"] = users1.user_description.str.replace("\n", " ")
users1["user_description"] = users1.user_description.str.replace(",", " ")
#users1.to_csv(f"{path_base}the_true_all_annotated_account.csv", sep = ",", index = False)
#users1 = users1.drop_duplicates().loc[users1["platform"]=="twitter"]
users1 = users1[['user_id', 'user_screen_name', 'user_name', 'Type_entite',
       'User_role', 'user_description', 'account_publication', 
       'Synthetic_logics']].drop_duplicates()#.reset_index()


In [ ]:
keep_all = True #Pour garder ou pas les comptes d'industries dont on a pas collecté les retweets

list_col = ['user_id', 'user_screen_name',
       'user_description', 'account_publication', 'Type_entite']

if keep_all == True:
    users1b = users1.merge(users0, on = list_col, how = "left")
    already_in_users1 = [x for x in users1b.user_id.unique()]
    users2 = users0.loc[~users0["user_id"].isin(already_in_users1)]
    #print(len(users), len(users0))
    
    users = pd.concat([users1b, users2], ignore_index=True)
else:
    users = users1.merge(users0, on = list_col, how = "left")
    


len(users)

In [ ]:
users.loc[(users["industry_verif"].isna()), "industry"]=False
users.loc[(users["industry_verif"] == True), "industry"]=True
users.loc[(users["industry"].isna()) | (users["industry"]==False) , "company"]="irrelevent"
users.loc[(users["industry"]==True) & (users["company"].isna()) , "company"]= "Unknown"



users = users.drop_duplicates(["user_screen_name","user_id","industry", "Type_entite"])

dict_logic = dict(zip(users.user_id, users.Synthetic_logics))
users["Synthetic_logics2"] = users.user_id.map(dict_logic.get)
users.loc[(users.industry==True), "Synthetic_logics2"] = "industry_pesticide"

dict_translation = {'industry_pesticide':'Industries pesticides', 
              'Agricultural Practices':'Pratiques agricoles',
       'Agroindustrial_perspectives':'Perspectives agroindustrielles', 
              'Comment_the_news':'Commentaires de l\'actualité',
       'Ecological_perspectives':'Perspectives écologiques', 
              'Health_perspectives':'Perspectives sanitaires',
       'Marketing_logic':'Perspectives commerciales', 
              'Pesticide_free_agriculture':'Lutte contre les pesticides',
       'Rationalist_perspectives':'Perspectives rationalistes'}

users["Synthetic_logicsfr"]= users.Synthetic_logics2.map(dict_translation.get)


In [ ]:
dict_logic2 = dict(zip(users.user_id, users.Synthetic_logics2))

#tweets["to_username"] = tweets.to_userid.map(dict(zip(users.user_id, users.user_screen_name)).get)

tweets["logic"] = tweets.user_id.map(dict_logic2.get)

tweets["company"] = tweets.user_id.map(dict(zip(users.user_id, users.company)).get)

tweets["industry"] = tweets.user_id.map(dict(zip(users.user_id, users.industry_verif)).get)

In [ ]:
tweets

In [ ]:
from search_regex import queries_in_dictionnary, texts_in_dictionnaries, search_regex

In [ ]:
dict_text = dict(zip(tweets.id, tweets.text.str.lower()))
queries = pd.read_csv(f"{aymeric}/search_queries/list_of_queries.csv", sep =",")


dict_queries = queries_in_dictionnary(data = queries, 
                                      column_name_of_queries = "nom_cluster.3", 
                                      list_of_regex = ["query_1"])

dict_queries

In [ ]:
d = search_regex(dict_queries = dict_queries, dict_text = dict_text)
d["somme"] = d.sum(axis=1)

In [ ]:
d["id"] = d.index
list_column = list(d.columns)[0:-2]
list_column

In [ ]:
d["logic"] = d.id.map(dict(zip(tweets.id, tweets.logic)).get)
d["industry"] = d.id.map(dict(zip(tweets.id, tweets.industry)).get)
d_industry = d.loc[d.industry==True]

In [ ]:
d1 = d.loc[(d.somme>0) & ~(d.logic.isna())]
d1_industry = d1.loc[d1.industry==True]
d1_industry

In [ ]:
d.logic.unique()

In [ ]:
d2 = d.loc[d.logic.isin(['Health_perspectives', 'Pesticide_free_agriculture',
       'Ecological_perspectives', 'Agroindustrial_perspectives','Comment_the_news',
       'Rationalist_perspectives',
       'industry_pesticide']) & (d.somme>0)]
d2.logic.unique()

In [ ]:
import numpy as np

In [ ]:
i=-1
for n, x in enumerate(d2.logic.unique()):
    i+=1
    list_size = []
    list_name_cluster= []
    for c in list_column:
        list_size.append(np.sum(d2[c].loc[d2.logic == x]))
        list_name_cluster.append(c)
    data = {"cluster": list_name_cluster, "nb":list_size}
    df0 = pd.DataFrame(data)
    df0["logic"] = x
    if i == 0:
        df = df0.copy()
    else:
        df = pd.concat([df,df0])
    


i = -1
df1 = tweets.loc[tweets.logic.isin(['Health_perspectives', 'Pesticide_free_agriculture',
       'Ecological_perspectives', 'Agroindustrial_perspectives','Comment_the_news',
       'Rationalist_perspectives',
       'industry_pesticide'])].groupby(["logic"])["logic"].count().reset_index(name='count')
df1

In [ ]:
df.loc[df.cluster.str.contains("Chlordécone")]

In [ ]:
freq_1 = df['nb'].loc[df.cluster=="Le vin"].values
freq_g1 = df1['count'].values
sum(freq_1)

In [ ]:
freq_g=freq_g1/sum(freq_g1)*sum(freq_1)
freq_g

In [ ]:
np.log(0)

In [ ]:
from scipy.stats import kstest

from numpy.random import seed
from numpy.random import poisson
from numpy.random import randn
from numpy.random import lognormal

from scipy.stats import chisquare


In [ ]:
statistics_topic={}
for topic in df["cluster"].unique():
    freq1  = df.loc[df["cluster"]==topic]
    freq_1=freq1['nb'].values
    if len(freq_1)==len(freq_g):
        ch=chisquare(f_obs=freq_1/sum(freq_1)*sum(freq_g), f_exp=freq_g)
        if ch.pvalue < .5:
            statistics_topic[topic]=ch.statistic
#            print(topic,ch)

In [ ]:
statistics_topic

In [ ]:
a1_sorted_keys = sorted(statistics_topic, key=statistics_topic.get, reverse=True)
for r in a1_sorted_keys:
    print(r, statistics_topic[r])

In [ ]:
df1_n=df1.copy()
df1_n['count'] = df1['count'] / sum(df1['count'])
df1_n

In [ ]:
for r in a1_sorted_keys:
    print(r)

In [ ]:
freq_all=pd.DataFrame()
for r in a1_sorted_keys:
    freq1_n=df.loc[df.cluster==r].copy()
    freq1_n['count'] = freq1_n['nb'] / sum(freq1_n['nb'])
    freq_duo=df1_n.merge(freq1_n,on='logic').rename(columns={'count_x':'global','count_y':'local'}).set_index('logic')
    freq_duo['zscore']=np.log(freq_duo['local']/freq_duo['global'])
    #del (freq_duo['local'])
    #del (freq_duo['global'])
    #del (freq_duo['nb'])
    print(freq_duo.columns)
    freq_all=pd.concat([freq_all,freq_duo],axis=0)
    freq_duo=df1_n.merge(freq1_n,on='logic').rename(columns={'count_x':'global','count_y':r}).set_index('logic')
    #freq_duo.plot(kind='bar')
    #plt.title("statistics= "+str(int(statistics_topic[r])))
    #plt.tight_layout()
    #plt.savefig('../image/freq_dist/'+r+'_'+category+'.png')

In [ ]:
list_excluded = [x for x in freq_all.cluster.loc[freq_all.zscore==-np.inf]]
list_excluded


In [ ]:
freq_all_cleaned = freq_all.replace(-np.inf, -10).reset_index()
freq_all_cleaned

In [ ]:
ct=pd.crosstab(freq_all_cleaned["cluster"], freq_all_cleaned.logic,values=freq_all_cleaned['zscore'],aggfunc="sum")
ct

In [ ]:
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm
rdgn = sns.diverging_palette(h_neg=130, h_pos=10, s=99, l=55, sep=3, as_cmap=True)
divnorm = TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)

fig=sns.clustermap(ct,method="centroid",cmap=rdgn, norm=divnorm,figsize=(10,20), 
                   dendrogram_ratio=(0.15,0.1), cbar_pos= (0.9, 0.8, 0.05, 0.18), mask = ct <= -10.0)

fig

In [ ]:
fig.savefig("image/content_analysis/clustermap_topic.png") 


In [ ]:
for x in tweets.text.loc[(tweets.logic=="industry_pesticide") & (tweets.text.str.lower().str.contains("sdhi"))]:
    print('########\n', x)

In [ ]:
import numpy as np
import re
import scipy
import statsmodels as sm
import statsmodels.stats
import statsmodels.stats.proportion

import statsmodels.api as sm
statsmodels.__version__

In [ ]:
list_nb_cluster = []
dict_nb_cluster = {}

log_or = []
log_upper = []
log_lower = []
world = []
list_theme = []

for x in list_column:
    nb_indus = np.sum(d1_industry[x])
    nb_indus_other_clust = np.sum(d1_industry["somme"]) - nb_indus
    nb_total = np.sum(d1[x])
    nb_other = nb_total - nb_indus
    nb_total_other = np.sum(d1["somme"]) - np.sum(d1_industry["somme"])
    nb_other_clust = nb_total_other - nb_other
    list_nb_cluster.append(nb_indus)
    #dict_nb_cluster[x] = {"indus": int(nb_indus), "non-indus":int(nb_other), "total":int(nb_total)}
    arr = np.array([[nb_indus, nb_indus_other_clust], 
                        [nb_other, nb_other_clust]])
    #print(arr)
        
    if 1:
                   table=statsmodels.stats.contingency_tables.Table2x2(arr)


                   logodd=table.log_oddsratio
                   [logodd_minus,logodd_plus]=table.log_oddsratio_confint(.95)
                   #print (logodd_minus,logodd_plus)
                   deviation=logodd
                   deviation_upper=logodd_plus
                   deviation_lower=logodd_minus
    log_or.append(deviation)
    log_upper.append(logodd_plus)
    log_lower.append(logodd_minus)
    list_theme.append(x)
    

    
#dict_nb_cluster

In [ ]:
data = {"theme":list_theme,"nb":list_nb_cluster,"log_or":log_or,"log_upper": log_upper, "log_lower":log_lower}
logor = pd.DataFrame(data).sort_values(["log_or"], ascending= True)
logor

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(1, figsize=(15,6), sharey = True, sharex=False)
#ax.set_xscale("log")

for n, i in enumerate(logor.theme):
        y = [i, i]
        x = [logor["log_lower"].iloc[n], logor["log_upper"].iloc[n]]

        ax.plot(x, y, "_-", markersize = 20, #color=couleur, 
                    markeredgewidth= 3, linewidth = 3)

        y = i
        x = logor["log_or"].iloc[n]
        ax.plot(x, y, "o", #color=couleur,
                markersize = 10)
        ax.yaxis.set_tick_params(labelcolor="black", labelsize=12)
        #ax.set_title(j, fontsize=14, fontweight='bold')
        ax.axvline(0, ls='--', linewidth=1, color='red')
        ax.set_xlabel("log odds ratio", fontsize=16)
        ax.set_ylabel("", fontsize=16)


In [ ]:
freq_all

In [ ]:
f = freq_all_cleaned.loc[freq_all_cleaned.logic=="industry_pesticide"]#.replace(-10.0, -np.inf)#.sort_values('zscore', ascending=False)
nb_cluster = dict(zip(freq_all_cleaned.cluster, freq_all_cleaned.nb))
logor["zscore"] = logor.theme.map(nb_cluster.get)



In [ ]:
sns.set_style("whitegrid")
f.loc[f.zscore < 0, "sur-représenté"] = "sous-représenté"
f.loc[f.zscore > 0, "sur-représenté"] = "sur-représenté"
fig = sns.barplot(f.sort_values("nb", ascending = False), x = "nb", y = "cluster", hue = "sur-représenté")

In [ ]:
d2 = d.loc[(d.somme>0) & ~(d.logic.isna())]
d2_industry = d2.loc[d2.industry==True]
list_nb_cluster = []
dict_nb_cluster = {}

log_or = []
log_upper = []
log_lower = []
world = []
list_theme = []

for x in list_column:
    nb_indus = np.sum(d2_industry[x])
    nb_indus_other_clust = np.sum(d2_industry["somme"]) - nb_indus
    nb_total = np.sum(d2[x])
    nb_other = nb_total - nb_indus
    nb_total_other = np.sum(d2["somme"]) - np.sum(d2_industry["somme"])
    nb_other_clust = nb_total_other - nb_other
    list_nb_cluster.append(nb_indus)
    #dict_nb_cluster[x] = {"indus": int(nb_indus), "non-indus":int(nb_other), "total":int(nb_total)}
    arr = np.array([[nb_indus, nb_indus_other_clust], 
                        [nb_other, nb_other_clust]])
    #print(arr)
        
    if 1:
                   table=statsmodels.stats.contingency_tables.Table2x2(arr)


                   logodd=table.log_oddsratio
                   [logodd_minus,logodd_plus]=table.log_oddsratio_confint(.95)
                   #print (logodd_minus,logodd_plus)
                   deviation=logodd
                   deviation_upper=logodd_plus
                   deviation_lower=logodd_minus
    log_or.append(deviation)
    log_upper.append(logodd_plus)
    log_lower.append(logodd_minus)
    list_theme.append(x)
    

    
#dict_nb_cluster

In [ ]:
data = {"theme":list_theme,"nb":list_nb_cluster,"log_or":log_or,"log_upper": log_upper, "log_lower":log_lower}
logor = pd.DataFrame(data).sort_values(["log_or"], ascending= True)

sns.set_style("whitegrid")
fig, ax = plt.subplots(1, figsize=(15,6), sharey = True, sharex=False)
#ax.set_xscale("log")

for n, i in enumerate(logor.theme):
        y = [i, i]
        x = [logor["log_lower"].iloc[n], logor["log_upper"].iloc[n]]

        ax.plot(x, y, "_-", markersize = 20, #color=couleur, 
                    markeredgewidth= 3, linewidth = 3)

        y = i
        x = logor["log_or"].iloc[n]
        ax.plot(x, y, "o", #color=couleur,
                markersize = 10)
        ax.yaxis.set_tick_params(labelcolor="black", labelsize=12)
        #ax.set_title(j, fontsize=14, fontweight='bold')
        ax.axvline(0, ls='--', linewidth=1, color='red')
        ax.set_xlabel("log odds ratio", fontsize=16)
        ax.set_ylabel("", fontsize=16)

In [ ]:
d.logic.unique()

In [ ]:
#d2 = d.loc[(d.somme>0) & ~(d.logic.isna())]
d2_agroind = d2.loc[d2.logic=='Pesticide_free_agriculture']
list_nb_cluster = []
dict_nb_cluster = {}

log_or = []
log_upper = []
log_lower = []
world = []
list_theme = []

for x in list_column:
    nb_indus = np.sum(d2_agroind[x])
    nb_indus_other_clust = np.sum(d2_agroind["somme"]) - nb_indus
    nb_total = np.sum(d2[x])
    nb_other = nb_total - nb_indus
    nb_total_other = np.sum(d2["somme"]) - np.sum(d2_agroind["somme"])
    nb_other_clust = nb_total_other - nb_other
    list_nb_cluster.append(nb_indus)
    #dict_nb_cluster[x] = {"indus": int(nb_indus), "non-indus":int(nb_other), "total":int(nb_total)}
    arr = np.array([[nb_indus, nb_indus_other_clust], 
                        [nb_other, nb_other_clust]])
    #print(arr)
        
    if 1:
                   table=statsmodels.stats.contingency_tables.Table2x2(arr)


                   logodd=table.log_oddsratio
                   [logodd_minus,logodd_plus]=table.log_oddsratio_confint(.95)
                   #print (logodd_minus,logodd_plus)
                   deviation=logodd
                   deviation_upper=logodd_plus
                   deviation_lower=logodd_minus
    log_or.append(deviation)
    log_upper.append(logodd_plus)
    log_lower.append(logodd_minus)
    list_theme.append(x)

In [ ]:
data = {"theme":list_theme,"nb":list_nb_cluster,"log_or":log_or,"log_upper": log_upper, "log_lower":log_lower}
logor = pd.DataFrame(data).sort_values(["log_or"], ascending= True)

sns.set_style("whitegrid")
fig, ax = plt.subplots(1, figsize=(15,6), sharey = True, sharex=False)
#ax.set_xscale("log")

for n, i in enumerate(logor.theme):
        y = [i, i]
        x = [logor["log_lower"].iloc[n], logor["log_upper"].iloc[n]]

        ax.plot(x, y, "_-", markersize = 20, #color=couleur, 
                    markeredgewidth= 3, linewidth = 3)

        y = i
        x = logor["log_or"].iloc[n]
        ax.plot(x, y, "o", #color=couleur,
                markersize = 10)
        ax.yaxis.set_tick_params(labelcolor="black", labelsize=12)
        #ax.set_title(j, fontsize=14, fontweight='bold')
        ax.axvline(0, ls='--', linewidth=1, color='red')
        ax.set_xlabel("log odds ratio", fontsize=16)
        ax.set_ylabel("", fontsize=16)